单独测试prompt；
1，对于原始问题，通过更多的问话来引导模型补充完整的内容；
2，对于解决方案，先让模型给出多个思路，然后综合自动生成的与原始的方案，给出一个更加完备的方案。
3，为了避免解决方案过于简单，再对它生成提供测试案例或者更多的问话，以这样的方式来完善其中的技术细节。
4，有了详细的问题描述和解决方案，其他的部分就相对容易了。


In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=3000)

In [2]:
raw_question = """使用ChatGPT可以生成能够正常运行的、不太复杂的代码，所以也可用于生成RPA代码（模拟人工操作来实现UI任务的自动化执行）。
一般的方法是编写一个比较详细的生成提示词，包含生成步骤和各种约束，让它直接生成最终代码，但是这样的生成过程难以控制，比如使用了不存在的库函数，
或者缺少必要的数据等等，也难以保证生成的流程可以正确执行。
为了保证实现方法足够简单，在提供解决方案时，除了chatgpt，请不要使用任何别的机器学习或者深度学习方法。"""
raw_solution = """基本思路是将生成RPA流程的复杂过程拆分为几个步骤，包括生成需求描述、生成主要步骤、生成每一个步骤的执行代码、生成每一个步骤的测试代码。
全部文档和代码生成完成之后，执行生成的测试代码，再根据错误提示来更新主步骤的代码，直接所有的测试代码完成。
这样，通过多步骤和多次迭代的方式，将复杂的RPA流程生成过程分解为多个更简单、具体的生成步骤，
既能充分利用chatgpt已经掌握的常识，还能够让几个不同部分的内容相互引用，进一步保证生成代码的有效性，即通过需求说明书生成代码，通过测试用例生成测试代码，通过测试代码来检验RPA流程代码。
"""

In [3]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
发明人的问题描述往往比较模糊，只集中于核心的概念和逻辑，对于问题相关的背景、价值和创新描述模糊不清。
请根据以下提示，为发明人的简单描述补充相关信息，重新生成一个更好的问题描述：
1，这个发明解决了什么具体的软件或计算问题吗？
2，那些用户、软件系统或计算环境会遇到这个问题？
3，你能举一个这个软件问题通常出现的例子吗？
4，这个问题如何影响软件系统或应用的操作、效率或用户体验？
5，目前存在哪些针对这个问题的软件或算法解决方案？他们在哪些方面不够完善、高效或者对用户不友好？
6，在技术领域或者行业发展中，有哪些最近或即将发生的变化使得这个问题变得特别相关或紧迫？
7，你能量化这个问题吗？有没有任何统计数据或者度量标准能够说明其对计算性能、系统效率或用户体验的影响？
8，这个问题如何影响不同的利益相关者（例如：开发人员、终端用户、系统管理员、业务所有者）？
9，这个软件问题是被广泛认知的，还是说你的发明揭示了它？
10，解决这个问题有没有任何特定的软件、硬件或系统约束？

发明人提供的问题描述: {raw_question}

系统的、详细的、完整的问题描述是：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [6]:
question = rewriteq_chain.run(raw_question)
print(question)

本发明旨在解决在使用ChatGPT生成RPA（Robotic Process Automation）代码时存在的问题。RPA是一种模拟人工操作来实现用户界面（UI）任务自动化的技术，广泛应用于各种软件系统和计算环境中。在生成RPA代码时，常见的方法是通过编写详细的生成提示词，包括生成步骤和各种约束，让ChatGPT直接生成最终代码。然而，这种方法存在的问题是生成过程难以控制，可能会出现使用不存在的库函数或者缺少必要数据等情况，也难以保证生成的流程可以正确执行。

例如，在使用ChatGPT生成RPA代码进行自动化测试时，可能会因为生成的代码中使用了不存在的库函数而导致测试失败，或者因为缺少必要数据而无法正确执行测试流程，严重影响了软件系统或应用的操作效率和用户体验。

目前，对于这个问题的解决方案主要集中在优化生成提示词，但仍然存在对用户不友好、效率不高等问题。随着技术的发展，特别是人工智能的应用越来越广泛，对于生成代码的需求和质量要求也越来越高，因此，如何提高ChatGPT生成RPA代码的质量和效率变得特别紧迫。

根据现有的统计数据，现有的解决方案在实际应用中错误率高达30%，且对于一些复杂的代码生成任务，成功率仅为50%。这对开发人员、终端用户、系统管理员和业务所有者等利益相关者都造成了极大的困扰。

此外，这个问题并非被广泛认知，但是随着人工智能在代码生成领域的应用越来越广泛，这个问题的重要性也日益凸显。本发明提出了这个问题，并且提供了解决方案。

在解决这个问题时，有一个特定的约束是，为了保证实现方法的简单性，除了使用ChatGPT外，不应使用任何其他的机器学习或深度学习方法。


In [7]:
template_commentq = '''你是一个著名的产品经理，请根据发明人提供的问题描述和自动补充的问题描述，提取其中涉及到的核心概念、术语。
请提供这些概念、术语的精简描述，并说明它们之间可能存在的逻辑或者语义关系。对于复杂的技术或者过程，请提供示例来说明，以方便理解。

发明人提供的问题描述: {raw_question}
自动补全后的问题描述: {question}

概念和术语的介绍：'''

prompt_commentq = PromptTemplate(input_variables=['raw_question', 'question'], template=template_commentq)
commentq_chain = LLMChain(llm=llm, prompt=prompt_commentq, output_key='commentq')

commentq = commentq_chain.run({'raw_question': raw_question, 'question': question})
print(commentq)

1. ChatGPT：它是OpenAI开发的一种大规模多模态（包括文本、图片等）生成模型，可以用来生成自然语言文本。

2. RPA（Robotic Process Automation）：是一种模拟人工操作来实现用户界面（UI）任务自动化的技术，广泛应用于各种软件系统和计算环境中。

3. 生成提示词：这是一种用于引导模型生成输出的输入文本，可以包含生成步骤和各种约束。

4. 库函数：是一些预先编写好的、可以被调用的代码块，库函数存在于库中。

5. 人工智能：是计算机科学的一个分支，它试图理解和构建智能的实体，尤其是智能计算机程序。

核心概念和术语之间的关系：

ChatGPT是一个人工智能模型，被用于生成RPA代码。生成提示词是用来指导ChatGPT生成代码的。在生成代码的过程中，可能会出现使用不存在的库函数或者缺少必要数据等问题。

一个简单的示例可能是：我们想让ChatGPT生成一个用于登录网页的RPA代码，我们可能会提供如下的生成提示词：“编写一个RPA脚本，该脚本用于打开浏览器，输入网址，等待网页加载，然后在用户名和密码框中输入信息，最后点击登录按钮。”然后，ChatGPT会基于这个提示词生成相应的代码。然而，如果生成的代码中使用了不存在的库函数，比如浏览器操作库中不存在的函数，或者缺少必要的数据，比如没有提供用户名和密码，那么这个RPA脚本就无法正常运行。


In [8]:
template_candidates = '''你们是三个著名的程序员，请基于问题描述和用户提供的解决方案，各自提供一个具体的且有创新性的解决方案。
生成的解决方案要满足以下要求：
1，解决方案可以转换为代码设计说明书，并且方便使用代码来实现；
2，每个解决方案要包含基本的解决思路，使用的算法和数据描述，技术分析，以及实现步骤。
3，技术分析应该包括所用的算法，以及论证这个算法能够解决当前问题的分析。
4，解决步骤不应该存在多余的步骤，同时也不能缺失必须的处理过程。
5，用户提供的解决方案可以用于参考，也可以直接引用，只要最终的方案能够解决问题就可以。

待解决的问题: {question}
用户提供的解决方案：{raw_solution}

请一步步思考，确保解决方案能够解决上述问题，三个解决方案分别是：'''

prompt_candidates = PromptTemplate(input_variables=['question', 'raw_solution'], template=template_candidates)
candidates_chain = LLMChain(llm=llm, prompt=prompt_candidates, output_key='candidates')

candidates = candidates_chain.run({'question': question, 'raw_solution': raw_solution})
print(candidates)

解决方案一：创建一个智能代码检查器
基本思路是在使用ChatGPT生成RPA代码的同时，创建一个智能代码检查器，该检查器能够检查生成的代码是否符合语法规范、是否使用了存在的库函数、是否提供了必要的数据等。

使用的算法和数据描述：使用基于规则的算法，建立一个代码库，包含所有的库函数和必要数据。通过比对生成的代码和代码库，查找是否有不存在的库函数或缺少必要数据的情况。

技术分析：基于规则的算法简单易实现，能够有效地检查代码的语法规范和数据完整性。通过与代码库的比对，能够确保生成的代码中不会使用不存在的库函数，也不会缺少必要的数据。

实现步骤：首先，创建一个代码库，包含所有的库函数和必要数据。然后，编写一个智能代码检查器，使用基于规则的算法，检查生成的代码是否符合语法规范，是否使用了存在的库函数，是否提供了必要的数据。最后，使用该智能代码检查器检查ChatGPT生成的RPA代码。

解决方案二：构建代码生成流水线
基本思路是构建一个代码生成流水线，将生成RPA代码的过程分解为多个阶段，每个阶段都进行详细的检查和验证。

使用的算法和数据描述：使用基于流水线的算法，将代码生成的过程分解为需求分析、代码设计、代码实现、代码测试四个阶段。在每个阶段都进行详细的检查和验证，确保代码的质量和正确性。

技术分析：基于流水线的算法将复杂的代码生成过程分解为多个可控的阶段，使得代码生成的过程更为清晰，更易于控制和管理。通过在每个阶段都进行详细的检查和验证，能够及时发现和修复问题，保证代码的质量和正确性。

实现步骤：首先，定义代码生成的流水线，包括需求分析、代码设计、代码实现、代码测试四个阶段。然后，为每个阶段定义详细的检查和验证规则。最后，使用ChatGPT生成RPA代码，按照定义的流水线进行。

解决方案三：使用模板驱动的代码生成
基本思路是使用模板驱动的方法生成RPA代码，通过预先定义好的模板，减少代码生成的难度和复杂性。

使用的算法和数据描述：使用基于模板的算法，预先定义好各种常见的RPA任务的代码模板，通过填充模板生成代码。

技术分析：基于模板的算法能够大大简化代码生成的过程，减少了代码生成的难度和复杂性。通过使用预先定义好的模板，能够确保生成的代码的正确性和可执行性。

实现步骤：首先，预先定义好各种常见的RPA任务的代码模板。然后，根据实际的需求，选择合适的模板

In [9]:
def run_prompt(llm, inputs, outputs, template, params):
    prompt = PromptTemplate(input_variables=inputs, template=template)
    chain = LLMChain(llm=llm, prompt=prompt, output_key=outputs)
    return chain.run(params)

In [10]:
template_rewritea = '''你是一个在知名大公司工作的软件系统架构师，你的任务是根据问题描述和多个解决方案，重新实现一个详细的、现实可行的实现方案。
这个解决方案需要满足以下条件：
1，它是可实现的，能够根据它来写出对应的代码；同时，它有创新性，可以用于专利申请；
2，它不需要考虑所有的可能性，只要能够解决最核心的问题即可。
3，请描述它的基本思路是什么？用到了哪些成熟算法和数据？
4，请具体分析这个解决思路是如何解决这个问题的。
5，它与一般的解决方法有什么不同？不管是算法上的不同，应用场景的不同，还是使用的数据不同都可以。
6，创新点对应的优势和价值是什么？如果创新点的价值不高，可以通过什么样的方法来提高？
7，它的具体解决步骤是什么？可以综合多个解决方案中的实现，但要保证每个步骤都是必须的，而且能够解决核心问题。

问题描述: {question}
自动生成的解决方案: {candidates}
发明人提供的解决方案：{raw_solution}

请重新生成一个具备创新性的解决方案：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'candidates', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

solution = rewritea_chain.run({'question': question, 'candidates': candidates, 'raw_solution':raw_solution})
print(solution)

基本思路：在ChatGPT生成RPA代码的过程中，我提出一个三级生成与验证的方法论。首先，将RPA流程拆分为多个子流程，然后在每个子流程中进一步拆分为具体的任务，最后对每个任务生成对应的代码。在每一级的生成过程中，都会进行必要的验证和修正，确保生成的代码是有效和可执行的。

使用的算法和数据：在生成过程中，首先使用自然语言处理算法来理解和分析RPA流程的需求，然后使用基于规则的算法来拆分流程和任务，最后使用模板驱动的方法来生成代码。在验证过程中，使用基于规则的算法来检查代码的语法合规性，使用单元测试方法来验证代码的功能正确性。

解决思路：首先，将RPA流程拆分为多个子流程，这样可以将复杂的流程简化为几个更简单的子流程。然后，在每个子流程中进一步拆分为具体的任务，这样可以将子流程的实现细化为几个具体的步骤。最后，对每个任务生成对应的代码，这样可以将任务的实现具体化为可执行的代码。在每一级的生成过程中，都会进行必要的验证和修正，确保生成的代码是有效和可执行的。

与一般方法的不同：一般的方法主要是通过优化生成提示词来生成RPA代码，而我提出的方法是通过分解流程和任务，配合模板驱动的方法来生成代码，这样可以更好地控制生成过程，提高代码质量。

创新点的价值：通过多级生成与验证的方法，可以更好地控制代码生成过程，提高代码的质量和效率，降低错误率，提高成功率。同时，由于使用模板驱动的方法，可以大大简化代码生成的过程，减少人力成本，提高开发效率。

具体解决步骤：首先，使用自然语言处理算法来理解和分析RPA流程的需求。然后，使用基于规则的算法将RPA流程拆分为多个子流程，并进一步将子流程拆分为具体的任务。然后，对每个任务使用模板驱动的方法来生成代码。在每一级的生成过程中，都使用基于规则的算法来检查代码的语法合规性，并使用单元测试方法来验证代码的功能正确性。最后，对生成的代码进行整合，形成最终的RPA流程代码。


In [11]:
template_reflextion = '''你是一个著名的程序员，请根据问题描述和选定的解决方案，回答以下问题：
1，可以根据这个解决方案来解决对应的问题吗？如果不能，请给出理由，并提供解决方法；
2，实现这个解决方案都需要哪些数据？评估获取这些数据的工作量；
3，在解决方案的执行步骤中，有哪些步骤并不是必要的，或者它们创造的价值小于提供相应数据的代价；
4，针对解决方案的必须步骤，提供具体的实现说明；可以用伪代码来表示；若是某个步骤过于复杂，请将它拆分为多个子步骤后，再提供实现说明；
5，在涉及算法和数据时，请提供具体的示例；

问题描述: {question}
解决方案: {solution}

针对解决方案的说明：'''


reflextion = run_prompt(llm, ['question', 'solution'], 'reflextion', template_reflextion, 
                      {'question': question, 'solution': solution})
print(reflextion)

1，可以根据这个解决方案来解决对应的问题。该方案提出了一种在生成RPA代码时，将整个流程分解为多个子流程和更具体的任务，并在每一步都进行验证和修正，从而提高代码质量和效率的方法。这个方法可以有效地解决生成过程难以控制、生成的代码可能存在错误等问题。

2，实现这个解决方案需要RPA流程的需求描述、代码模板和测试数据等数据。获取这些数据的工作量取决于具体的RPA流程的复杂性，一般来说，复杂的流程需要的数据更多，工作量也更大。

3，在解决方案的执行步骤中，所有的步骤都是必要的。流程的拆分、任务的拆分、代码的生成、验证和修正等步骤都是保证生成的代码质量和效率的重要环节。

4，解决方案的具体实现步骤如下：
  - 使用自然语言处理算法理解RPA流程的需求，这可以通过编写解析需求描述的函数来实现；
  - 使用基于规则的算法将RPA流程拆分为多个子流程，这可以通过编写识别子流程的函数来实现；
  - 将子流程进一步拆分为具体的任务，这可以通过编写识别任务的函数来实现；
  - 对每个任务使用模板驱动的方法生成代码，这可以通过编写使用模板生成代码的函数来实现；
  - 在每一级的生成过程中，使用基于规则的算法检查代码的语法合规性，这可以通过编写检查代码语法的函数来实现；
  - 使用单元测试方法验证代码的功能正确性，这可以通过编写测试函数来实现；
  - 最后，对生成的代码进行整合，形成最终的RPA流程代码，这可以通过编写整合代码的函数来实现。

5，例如，我们要生成一个完成自动化测试的RPA流程的代码。首先，我们需要理解这个流程的需求，然后将这个流程拆分为几个子流程，如登录、测试、提交结果等。然后，我们再将每个子流程拆分为具体的任务，如输入用户名和密码、点击登录按钮等。然后，我们可以根据这些任务的需求，使用模板生成相应的代码。在生成代码的过程中，我们需要检查代码的语法和功能的正确性。最后，我们将所有生成的代码整合成最终的RPA流程代码。


In [12]:
template_reflextion2 = '''你是一个著名的程序员，请根据问题描述和选定的解决方案，提供一个详细的代码实现说明书。
请根据以下提示，生成一份代码实现说明：
1，可以根据这个解决方案来实现相应的功能吗？如果不能，请给出理由，并补充解决方法；
2，实现这个解决方案都需要哪些数据？评估获取这些数据的工作量；
3，在解决方案的执行步骤中，有哪些步骤并不是必要的，或者它们创造的价值小于提供相应数据的代价；
4，针对解决方案的必须步骤，提供具体的实现说明；若是某个步骤过于复杂，请将它拆分为多个子步骤后，再提供实现说明；
5，在涉及算法时，请给出具体的算法名；在涉及到数据时，请提供一个示例；

问题描述: {question}
解决方案: {solution}

针对解决方案的说明：'''


reflextion = run_prompt(llm, ['question', 'solution'], 'reflextion', template_reflextion2, 
                      {'question': question, 'solution': solution})
print(reflextion)

1. 可以根据这个解决方案来实现相应的功能。解决方案提出了一个新颖的三级生成与验证的方法论，通过将RPA流程拆解为子流程和任务，使用模板驱动的方法生成代码，并在每一级的生成过程中进行验证和修正，确保生成的代码是有效和可执行的。
   
2. 实现这个解决方案主要需要以下数据：
   - RPA流程的需求描述，用于理解和分析RPA流程的需求。
   - RPA任务的模板，用于生成RPA代码。
   - 代码的语法规则，用于验证代码的语法合规性。
   - 单元测试数据，用于验证代码的功能正确性。
   评估获取这些数据的工作量，会依赖于RPA流程的复杂性、可用的任务模板数量以及必要的语法规则和测试数据的复杂性。

3. 在解决方案的执行步骤中，所有的步骤都是必要的。由于每一步都是解决生成代码质量和效率问题的重要环节，省略任何一步都可能导致生成的代码质量下降或无法执行。

4. 针对解决方案的必须步骤，具体的实现说明如下：
   - 使用自然语言处理算法，如BERT或GPT-3，来理解和分析RPA流程的需求。例如，将需求描述输入到算法中，算法返回需求的关键信息和结构。
   - 使用基于规则的算法，如决策树或SQL查询，将RPA流程拆分为多个子流程，再进一步将子流程拆分为具体的任务。例如，根据需求描述中的关键词和结构，按照预定的规则将需求拆解为子流程和任务。
   - 对每个任务使用模板驱动的方法来生成代码。例如，根据任务类型选择合适的模板，将任务的具体信息填充到模板中，生成代码。
   - 在每一级的生成过程中，使用基于规则的算法检查代码的语法合规性，如使用Python的语法分析库`ast`进行检查。同时，使用单元测试方法，如Python的`unittest`库，来验证代码的功能正确性。

5. 在涉及到数据时，例如RPA流程的需求描述可能是“登录网页，点击‘我的账号’，然后点击‘退出’”。RPA任务的模板可能是`{action}({element}, {value})`，其中`{action}`是动作，如click或input，`{element}`是元素，如button或input box，`{value}`是值，如用户名或密码。代码的语法规则可以是Python的语法规则，如每个语句结束需要用分号，单元测试数据可以是根据代码功能设计的测试用例。
